In [1]:
import pandas as pd
import datetime
import os
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
ratio_dist=25

In [2]:
input_locations=pd.ExcelFile("./sampleevents.xlsx")
input_locations.sheet_names

['sampleevents', 'Sheet1']

In [3]:
# Still "Sheet1" sheet
input_locations=input_locations.parse("Sheet1",dtype=str)
input_locations.head(2)

,locale,venue_id,venue_name,venue_city,venue_state,venue_postalCode,venue_country,venue_venueConfigId,venue_venueConfigName,ticketInfo_minListPrice,ticketInfo_maxListPrice,ticketInfo_totalTickets,ticketInfo_totalListings
0,en_US,6124,The Colosseum at Caesars Palace,Las Vegas,NV,89109,US,1294179,All Rsvd Sections 100-400 - Dynamic MASTER,96,2571.52,1309,266
1,en_US,183459,Fontainebleau Hotel and Resort,Miami Beach,FL,33140,US,1243460,GA and VIP Tables - Dynamic,0,0,0,0


In [4]:
df_TA=input_locations[['venue_id','venue_name','venue_city','venue_state','venue_postalCode','venue_country']].drop_duplicates()
df_TA.shape,df_TA['venue_id'].nunique()

((2, 6), 2)

In [5]:
df_TA['search_key']=df_TA.apply(lambda x: ", ".join(list([x['venue_name'],x['venue_city'],x['venue_state'],x['venue_postalCode'],x['venue_country']])),axis=1)


In [6]:
location_input_dict=df_TA.set_index("venue_id").to_dict()['search_key']
location_input_dict

{'183459': 'Fontainebleau Hotel and Resort, Miami Beach, FL, 33140, US',
 '6124': 'The Colosseum at Caesars Palace, Las Vegas, NV, 89109, US'}

In [7]:
# Manully copied here through browser

dict_store_locations={}

# '183459': 'Fontainebleau Hotel and Resort, Miami Beach, FL, 33140, US'
dict_store_locations.update({"183459":{"Google_Address":"4441 Collins Ave, Miami Beach, FL 33140","Google_Coor":[25.8169812,-80.1225072]}})
# '6124': 'The Colosseum at Caesars Palace, Las Vegas, NV, 89109, US'
dict_store_locations.update({"6124":{"Google_Address":"3570 S Las Vegas Blvd, Las Vegas, NV 89109","Google_Coor":[36.117914,-115.1750347]}})


In [8]:
df_TA['Google_Address']=df_TA['venue_id'].apply(lambda x: dict_store_locations[x]['Google_Address'])
df_TA['Google_Coordinate']=df_TA['venue_id'].apply(lambda x: dict_store_locations[x]['Google_Coor'])

In [10]:
df_TA=df_TA.reset_index()
del df_TA['index']

df_store_zip=pd.DataFrame()

for ind,row in df_TA.iterrows():
    store_center=row['Google_Coordinate']
    zip_list=[]
    store_zip=pd.DataFrame()
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi") # unit="mi"
        if dist<=ratio_dist:
            zip_list.append(zip_cd)
            df=pd.DataFrame({"dist_miles":dist},index=[zip_cd])
            store_zip=store_zip.append(df)
    store_zip=store_zip.reset_index().rename(columns={"index":"zip_cd"})
    store_zip['venue_id']=row['venue_id']
    df_TA.loc[ind,'zip_list']=str(zip_list)
    df_store_zip=df_store_zip.append(store_zip)

In [11]:
df_TA['zip_list']=df_TA['zip_list'].apply(eval)

In [13]:
set(df_TA.loc[0,'zip_list']).intersection(set(df_TA.loc[1,'zip_list'])) # No obvious overlap

set()

In [14]:
temp_df=input_locations[['venue_id','venue_name','venue_city','venue_state','venue_postalCode','venue_country']].drop_duplicates()
df_store_zip=pd.merge(temp_df,df_store_zip,on="venue_id",how="left")
df_store_zip.shape

(260, 8)

In [15]:
df_store_zip['zip_cd'].nunique()

260

In [16]:
writer=pd.ExcelWriter("./Trade_area_for_2_venue_locations_25_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_store_zip.to_excel(writer,"by_store_zip",index=False)
df_TA.to_excel(writer,"2_locations",index=False)
writer.save()